# Dev Notes

Create & save new Influx DB measures:
- VerticalLine
- HorizontaLine

These will need to include start and stop variables so that we don't have lines into perpetuity

In [1]:
# Required for PydanticAI to work with Jupyter (nested event loops)
import nest_asyncio

nest_asyncio.apply()

In [2]:
import logging
import asyncio
import pandas as pd
import polars as pl
from zoneinfo import ZoneInfo

from datetime import datetime, timedelta, timezone

import influxdb_client

from tastytrade.common.logging import setup_logging

from tastytrade.connections import InfluxCredentials

from tastytrade.config import RedisConfigManager
from tastytrade.providers.market import MarketDataProvider
from tastytrade.providers.subscriptions import RedisSubscription

from tastytrade.messaging.models.events import CandleEvent

from tastytrade.analytics.visualizations.plots import (
    plot_macd_with_hull,
    HorizontalLine,
    VerticalLine,
)
from tastytrade.analytics.indicators.momentum import macd
import re

from tastytrade.analytics.visualizations.utils import get_opening_range

import pytz

In [3]:
# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

2025-08-10 08:20:23 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250810.log


# Market Data Subscriptions

In [4]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize()

influxdb = influxdb_client.InfluxDBClient(
    url=InfluxCredentials(config=config).url,
    token=InfluxCredentials(config=config).token,
    org=InfluxCredentials(config=config).org,
)

subscription = RedisSubscription(config=RedisConfigManager())
await subscription.connect()

streamer = MarketDataProvider(subscription, influxdb)

2025-08-10 08:20:23 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2025-08-10 08:20:23 - INFO:tastytrade.providers.subscriptions:72:Listening to Redis at redis://redis:6379/0


# Date Setup

In [5]:
datetime.now()

datetime.datetime(2025, 8, 10, 8, 20, 23, 469375)

In [6]:
year = datetime.now().year

month = datetime.now().month
day = 7
day = 7  # datetime.now().day

et_tz = pytz.timezone("America/New_York")
et_tz = ZoneInfo("America/New_York")

market_open = datetime(year, month, day, 9, 30, tzinfo=et_tz)
morning_end = datetime(year, month, day, 11, 30, tzinfo=et_tz)
lunch_end = datetime(year, month, day, 13, 30, tzinfo=et_tz)
market_close = datetime(year, month, day, 16, 0, tzinfo=et_tz)

streamer = MarketDataProvider(subscription, influxdb)

start = market_open.astimezone(timezone.utc) - timedelta(minutes=30)
stop = market_close.astimezone(timezone.utc)

In [7]:
candle_symbol = "SPX{=m}"

prior_day: CandleEvent = CandleEvent(
    **(
        streamer.download(
            symbol=re.sub(r"\{=.*?\}", "{=d}", candle_symbol),
            start=market_open.date() + timedelta(days=-1),
            stop=market_open.date(),
            debug_mode=True,
        )
        .to_dicts()
        .pop()
    )
)

In [8]:
or5 = await get_opening_range(
    streamer,
    "SPX{=m}",
    5,
    date=start.date(),
)

or15 = await get_opening_range(
    streamer,
    "SPX{=m}",
    15,
    date=start.date(),
)

or30 = await get_opening_range(
    streamer,
    "SPX{=m}",
    30,
    date=start.date(),
)

2025-08-10 08:20:23 - INFO:tastytrade.analytics.visualizations.utils:92:Opening range for SPX{=m} (5m): High=6388.02, Low=6374.32
2025-08-10 08:20:23 - INFO:tastytrade.analytics.visualizations.utils:92:Opening range for SPX{=m} (15m): High=6389.71, Low=6374.32
2025-08-10 08:20:23 - INFO:tastytrade.analytics.visualizations.utils:92:Opening range for SPX{=m} (30m): High=6389.71, Low=6373.34


In [9]:
levels = [
    HorizontalLine(
        price=prior_day.close,
        color="#FF66FE",  # Orange
        line_dash="dot",
        label_font_size=10.5,
        label="prior close",
        opacity=0.45,
        end_time=market_close,
    ),
    HorizontalLine(
        price=prior_day.high,
        color="#4CAF50",  # Green
        line_dash="dot",
        label_font_size=10.5,
        label="prior high",
        opacity=0.45,
        end_time=market_close,
    ),
    HorizontalLine(
        price=prior_day.low,
        color="#F44336",  # Red
        line_dash="dot",
        label_font_size=10.5,
        label="prior low",
        opacity=0.45,
        end_time=market_close,
    ),
    HorizontalLine(
        price=or5.high,
        start_time=market_open,
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
        # label="5min hi",
    ),
    HorizontalLine(
        price=or5.low,
        start_time=market_open,
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.75,
    ),
    HorizontalLine(
        price=or15.high,
        start_time=market_open + timedelta(minutes=15),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.high != or5.high else 0.0,
    ),
    HorizontalLine(
        price=or15.low,
        start_time=market_open + timedelta(minutes=15),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="solid",
        opacity=0.45 if or15.low != or5.low else 0.0,
    ),
    HorizontalLine(
        price=or30.high,
        start_time=market_open + timedelta(minutes=30),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.high != or15.high else 0.0,
    ),
    HorizontalLine(
        price=or30.low,
        start_time=market_open + timedelta(minutes=30),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.low != or15.low else 0.0,
    ),
    HorizontalLine(
        price=or30.low,
        start_time=market_open + timedelta(minutes=30),
        end_time=market_close,
        color="#4CAF50",  # Green
        line_dash="dot",
        opacity=0.45 if or30.low != or15.low else 0.0,
    ),
    # HorizontalLine(
    #     price=5903,
    #     start_time=datetime(2025, 5, 29, 9, 40, tzinfo=et_tz),
    #     color="#555555",  # Green
    #     line_dash="dot",
    #     label="Bearish",
    #     # opacity=0.45 if or30.low != or15.low else 0.0,
    # ),
    # HorizontalLine(
    #     price=5893,
    #     start_time=datetime(2025, 5, 29, 11, 30, tzinfo=et_tz),
    #     color="#555555",  # Green
    #     line_dash="dot",
    #     label="Bullish",
    #     # opacity=0.45 if or30.low != or15.low else 0.0,
    # ),
    # HorizontalLine(
    #     price=5432,
    #     start_time=datetime(2025, 4, 15, 10, 17, tzinfo=et_tz),
    #     color="#555555",  # Green
    #     line_dash="dot",
    #     opacity=1,
    # ),
]

In [10]:
executions = [
    VerticalLine(
        time=datetime(year, month, day, 14, 30, tzinfo=et_tz),
        color="#555555",
        line_dash="dot",
        label="Open",
    ),
    # VerticalLine(
    #     time=datetime(2025, 2, 26, 14, 30) + timedelta(hours=4, minutes=50),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Close",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 2, 27, 14, 30) + timedelta(minutes=20),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 2, 28, 14, 30) + timedelta(hours=1, minutes=40),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 2, 28, 14, 30) + timedelta(hours=4, minutes=0),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Close",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 3, 14, 30) + timedelta(hours=0, minutes=20),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 4, 14, 30) + timedelta(hours=1, minutes=10),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Pass",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 4, 14, 30) + timedelta(hours=2, minutes=10),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 6, 14, 30) + timedelta(hours=0, minutes=45),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Pass",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 6, 14, 30) + timedelta(hours=2, minutes=10),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 7, 14, 30) + timedelta(hours=0, minutes=45),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 7, 14, 30) + timedelta(hours=2, minutes=55),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 13, 10, 20, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 13, 10, 45, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 18, 10, 40, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 19, 10, 35, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 20, 11, 34, 32, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 20, 12, 15, 8, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Close",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 3, 20, 13, 9, 34, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 4, 1, 10, 35, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 4, 8, 10, 42, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 4, 15, 10, 17, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 5, 29, 10, 11, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Open",
    # ),
    # VerticalLine(
    #     time=datetime(2025, 5, 29, 12, 11, tzinfo=et_tz),
    #     color="#555555",
    #     line_dash="dot",
    #     label="Close",
    # ),
]

In [11]:
candles: pl.DataFrame = streamer.download(
    symbol=candle_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

df_macd = macd(
    candles, prior_close=prior_day.close, fast_length=12, slow_length=26, macd_length=9
)

plot_macd_with_hull(
    df_macd,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop + timedelta(minutes=15),
    horizontal_lines=levels,
    vertical_lines=executions,
)

In [12]:
candles_5m: pl.DataFrame = streamer.download(
    symbol=candle_symbol.replace("m", "5m"),
    start=start,
    stop=stop,
    debug_mode=True,
)

df_macd_5m = macd(
    candles_5m,
    prior_close=prior_day.close,
    fast_length=12,
    slow_length=26,
    macd_length=9,
)

plot_macd_with_hull(
    df_macd_5m,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop + timedelta(minutes=15),
    horizontal_lines=levels,
    vertical_lines=executions,
)